In [21]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = "1xnMYOD5FkrwVKakAAaAaqOyD"
        consumer_secret = "hf1ADLyEPWll1rl43sGHdwl7eMWmK1LbzU6ajSxQKbwB86I15D"
        access_token = "732181352326434816-XpBDVRPACxdBtChDFqxVhH2bz5nRGm0"
        access_token_secret = "2tu6wcnVBTy6aZNtNjYrSmzzJv1pAvGI2YbTNgjXPc8rv"
        

        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")

    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return (' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()))

    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []

        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)

            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}

                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)

            # return parsed tweets
            return tweets

        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))




In [76]:
import pandas as pd
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    
    myFilters = ["cash flow", "balance sheet", "stock", "prices", "deprecation"]
    ptweets = []
    ntweets = []
    neuTweets = []
    
    myDf = pd.DataFrame()
    for currentFilter in myFilters:
        # calling function to get tweets
        tweets = api.get_tweets(query = currentFilter, count = 2000)

        # picking positive tweets from tweets
        print(len(ptweets),len(ntweets),len(neuTweets))
        if len(ptweets) == 0:
            ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
        else:
            ptweets.append([tweet for tweet in tweets if tweet['sentiment'] == 'positive'])
        # percentage of positive tweets
        print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
        # picking negative tweets from tweets
        
        if len(ntweets) == 0:
            ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
        else:
            ntweets.append([tweet for tweet in tweets if tweet['sentiment'] == 'negative'])
        # percentage of negative tweets
        print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))

        neut = [tweets not in ptweets]
        
        if len(neuTweets) == 0:
            neuTweets = [neut not in ntweets]
        else:
            neuTweets.append([neut not in ntweets])
        
        print(len(neuTweets))
        # percentage of neutral tweets
        #print("Neutral tweets percentage: {} %".format(100*len(tweets - ntweets - ptweets)/len(tweets)))

        # printing first 5 positive tweets
        #print("\n\nPositive tweets:")
        #for tweet in ptweets[:10]:
        #    print(tweet['text'])

        # printing first 5 negative tweets
        #print("\n\nNegative tweets:")
        #for tweet in ntweets[:10]:
        #    print(tweet['text'])
    
    a = [t for t in ptweets]
    print(a)
    myDf = pd.DataFrame([[t['text'],3] for t in ptweets],columns=['tweet','label'])
    myDf = myDf.append(pd.DataFrame([[t['text'],3] for t in ntweets],columns=['tweet','label']))
    myDf = myDf.append(pd.DataFrame([[t['text'],3] for t in neuTweets],columns=['tweet','label']))
   
    #myDf['label'].iloc[currentLen:]  = 2
    currentLen = len(myDf)
    
    
    return (myDf,ptweets)

#print(ptweets)
#myDf['tweets'] = ptweets
#myDf['label']  = 3


myDf,ptweets = main()

0 0 0
Positive tweets percentage: 33.75 %
Negative tweets percentage: 20.0 %
1
27 16 1
Positive tweets percentage: 38.888888888888886 %
Negative tweets percentage: 23.61111111111111 %
2
28 17 2
Positive tweets percentage: 36.25 %
Negative tweets percentage: 22.5 %
3
29 18 3
Positive tweets percentage: 31.914893617021278 %
Negative tweets percentage: 20.21276595744681 %
4
30 19 4
Positive tweets percentage: 35.632183908045974 %
Negative tweets percentage: 22.988505747126435 %
5
[{'text': 'RT @MOMOSTOCKTRADES: $DIGAF New ICO offering to fund operations and create cash flow. Im betting on the #Bitcoin madness that will make thi…', 'sentiment': 'positive'}, {'text': 'Large-cap growth stocks with more compelling valuation. three growth companies with shares that are attractively pr… https://t.co/8yg0d4BntL', 'sentiment': 'positive'}, {'text': '@PeckPolitics @HeidiL_RN But they’ll give up some of that cash flow to gain more power.  I get your point.  It’s qu… https://t.co/0pvomPRHs5', 'senti

TypeError: list indices must be integers or slices, not str

In [30]:
!pip install tweepy
!pip install textblob

In [55]:
myDf['tweets'].iloc[0]

{'sentiment': 'positive',
 'text': 'RT @MOMOSTOCKTRADES: $DIGAF New ICO offering to fund operations and create cash flow. Im betting on the #Bitcoin madness that will make thi…'}

In [56]:
len(myDf)

32

In [61]:
pd.DataFrame({"sahil",2},columns=['tweets','label'])

,tweets,label
0,sahil,2
